In [24]:
# initial imports
import numpy as np
import pylab as plt
import os
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils import CircularAperture,SkyEllipticalAperture,aperture_photometry,EllipticalAnnulus
from photutils.utils import calc_total_error
from astropy.coordinates import SkyCoord
from astropy.stats import sigma_clipped_stats
from astropy import wcs
import glob
import cosmo
import math

In [25]:
snid = '1334084'
ra = 41.734005
dec = -0.690615

In [26]:
files = ['/Users/hsolak/Desktop/DESJ024656.2-004126.2_g.fits']

# for each image, get obs filter and image coordinates for the SN
filters,xpos,ypos = [],[],[]
for f in files:
    print(f)
    header = fits.getheader(f)
    filters += [header['BAND']]  # changed this from [fits.getval(f,'FPA.FILTER').split('.')[0]] because it gave an error
    ps1_wcs = wcs.WCS(fits.getheader(f))
    snx,sny = ps1_wcs.wcs_world2pix([(ra,dec)],0)[0]
    xpos += [snx]; ypos += [sny]

/Users/hsolak/Desktop/DESJ024656.2-004126.2_g.fits


In [27]:
apr_arcsec = 1 #(~1 FWHM)
# arcsec to number of PS1 pixels (0.25 arcsec/pix)
apr_pix = apr_arcsec/0.25

In [28]:
## routine for doing the aperture photometry
## note that no sky subtraction is done because PS1 images should already be sky-subtracted
def doAperPhot(xpos,ypos,image,exptime,zpt,aprad,errorim=[],unit='counts/sec'):

    if type(xpos) == np.ndarray:
        positions = [(x,y) for x,y in zip(xpos,ypos)]
    else:
        positions = [(xpos,ypos)]

    if not len(errorim):
        # get the uncertainty from the sky standard deviation, sigma-clipped STD
        avg,sky,skystd = sigma_clipped_stats(
            image[(image != 0)],
            sigma=3.0)

    ap = CircularAperture(positions,r=aprad)

    # this is a big weird, but part of checking if aperture extends beyond image
    apermask = ap.to_mask(method='exact')[0]
    imshape = np.shape(image)
    apermask_full = apermask.to_image(imshape)
    apermask_cols = np.where(apermask_full > 0)
    if apermask.bbox.ixmin < 0 or apermask.bbox.iymin < 0 or \
       apermask.bbox.ixmax > imshape[1]-1 or apermask.bbox.iymax > imshape[0]-1:
        print('Error : aperture extends outside the image!!')
        if type(xpos) == np.ndarray:
            return(np.array([np.nan]*len(xpos)),np.array([np.nan]*len(xpos)),np.array([np.nan]*len(xpos)),
                   np.array([np.nan]*len(xpos)),np.array([1]*len(xpos)))
        else:
            return(np.nan,np.nan,np.nan,np.nan,1)
    try:
        apOutsideImageCols = np.where((apermask_cols[0] == 0) | (apermask_cols[1] == 0) |
                                      (apermask_cols[0] == imshape[0]-1) | (apermask_cols[1] == imshape[1]-1))[0]
    except:
        import pdb; pdb.set_trace()

    # check if aperture extends outside the image
    if len(apOutsideImageCols):
        print('Error : aperture extends outside the image!!')
        if type(xpos) == np.ndarray:
            return(np.array([np.nan]*len(xpos)),np.array([np.nan]*len(xpos)),
                   np.array([sky]*len(xpos)),np.array([np.nan]*len(xpos)),np.array([1]*len(xpos)))
        else:
            return(np.nan,np.nan,sky,np.nan,1)

    if len(errorim):
        phot_table = aperture_photometry(image, ap, error=errorim)
    else:
        # calc error from sky standard deviation and image data (poisson noise & sky std)
        if unit == 'counts/sec':
            err = calc_total_error(image, skystd, exptime)
        else:
            err = calc_total_error(image, skystd, 1.0)
        phot_table = aperture_photometry(image, ap, error=err)

    if type(xpos) == np.ndarray:
        flux = phot_table['aperture_sum']	
        fluxerr = phot_table['aperture_sum_err']#**2.#+ skystd**2./ellap.area() )
    else:
        flux = phot_table['aperture_sum'][0]	
        fluxerr = phot_table['aperture_sum_err'][0]#**2.#+ skystd**2./ellap.area() )

    mag = -2.5*np.log10(flux) + zpt
    magerr = 2.5/np.log(10)*fluxerr/flux

    return(mag,magerr,flux,fluxerr,0)

In [29]:
## let's do some aperture photometry!
for f,x,y,flt in zip(files,xpos,ypos,filters):
        image = fits.getdata(f)
        header = fits.getheader(f)
        dr1_zpt = header['MAGZERO']
        zpt = 10**(-0.4*(dr1_zpt-27.5))
        #print('ps1-zpt',ps1_zpt)
        #print(zpt)
        # PS1 zeropoints are set to 25 + 2.5*log10(exptime)
        mag,magerr,flux,fluxerr,badflag = \
            doAperPhot(x,y,image,header['EXPTIME'],dr1_zpt,
                       apr_pix,unit='counts')
        # wikipedia says surface brightness is mag + 2.5*np.log10(area)
        sb = mag + 2.5*np.log10(np.pi*apr_arcsec**2.)
        flux = flux * zpt / math.pi
        fluxerr = fluxerr * zpt / math.pi
        
        if flt == 'g':
            print('g flux =',flux)
            print('g sb =',sb)

g flux = 293.2520992494893
g sb = 21.331897175744583


In [30]:
# From the measurements file:
# 1334084 41.734005 -0.690615 186.36 226.0901460804622 21.61429591386391 0.012031912404473159
# sb = 21.61429591386391

21.331897175744583 - 21.61429591386391

-0.28239873811932625